In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np, pandas as pd, os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader



In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


In [ ]:
train = pd.read_csv('/kaggle/input/instant-gratification/train.csv')
test = pd.read_csv('/kaggle/input/instant-gratification/test.csv')

In [ ]:
train.head()

In [ ]:
cols = [c for c in train.columns if c not in ['id', 'target']]
oof = np.zeros(len(train))
skf = StratifiedKFold(n_splits=5, random_state=42)

In [ ]:
train['target']

In [ ]:
cols

In [ ]:
len(cols)

In [ ]:
### define our neural network
class IGClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.activation = nn.ELU(inplace=True)  ### inplace operation is faster -- non-linearity
        self.fc1 = nn.Linear(len(cols), 512)  ### 1st fully connected/linear layer  -- note the dimensions
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 512)
        self.bn2 = nn.BatchNorm1d(512)        ### scaling layer inputs to mean = 0 std = 1; generally used after non-linearity
        self.fc3 = nn.Linear(512, 1)
        self.drop = nn.Dropout(0.2)           ### dropout is regularization function that drops x% of neurons in a layer 
    def forward(self, x):
        data = x
        out = (self.bn1(self.activation(self.fc1(data))))    
        out = (self.bn2(self.activation(self.fc2(out))))
        out = self.fc3(out)
        return out

In [ ]:
from torch.optim.optimizer import Optimizer
debug =False

In [ ]:
oof = np.zeros(len(train))
preds = np.zeros(len(test))
batch_size = 2048
n_epochs = 10

In [ ]:
### define k fold
skf = StratifiedKFold(n_splits=20, random_state=42)

### bring test dataset to GPU by doing .cuda()
x_test_cuda = torch.tensor(test[cols].values, dtype=torch.float).cuda()  

#### dataloader for test set based on BS
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test_cuda), batch_size=batch_size, shuffle=False)

In [ ]:
idx1 = train.index; idx2 = test.index
scheduler = False
fold_= 1
    
for train_index, test_index in skf.split(train[cols], train['target']):
    x_train_fold = torch.tensor(train.loc[train_index][cols].values, dtype=torch.float).cuda()
    y_train_fold = torch.tensor(train.loc[train_index]['target'].values[:, np.newaxis], dtype=torch.float32).cuda()
    x_val_fold = torch.tensor(train.loc[test_index][cols].values, dtype=torch.float).cuda()
    y_val_fold = torch.tensor(train.loc[test_index]['target'].values[:, np.newaxis], dtype=torch.float32).cuda()

    loss_fn = torch.nn.BCEWithLogitsLoss()   ### with logits because our model outputs raw logits or WtX+b values
    model = IGClassifier()
    model.cuda()
    
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0005,weight_decay=1e-5) # Using Adam optimizer
    step_size, base_lr, max_lr = 10, 0.0005, 0.0008  ### ignore for now
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=max_lr)
    #scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr,step_size=step_size, mode='exp_range',gamma=0.99994)
    
    train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train_fold, y_train_fold), batch_size=batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_val_fold, y_val_fold), batch_size=batch_size, shuffle=False)
    avg_losses_f = []
    avg_val_losses_f = []
    best_auc = 0
    earlystop = 0
    test_preds_fold = np.zeros((len(test)))
    
    for epoch in range(n_epochs):   ### iterate over n-epochs; note that we do total of n_epochs*kfolds runs over entire data
        model.train()  ### start training
        avg_loss = 0.  ### train loss init
        
        for i, (x_batch, y_batch) in enumerate(train_loader): ### loop over all batches of training data 
            y_pred = model(x_batch)   ### get preds
            if scheduler:
                scheduler.batch_step()
            loss = loss_fn(y_pred, y_batch)  ### calculate loss
            optimizer.zero_grad()            ### zero gradient
            loss.backward()                  ### backward prop
            optimizer.step()                 ### adjust weights
            avg_loss += loss.item()/len(train_loader) ### calculate train avg. loss
        
        
        model.eval()  ### start evaluation mode to get validation losses
        valid_preds_fold = np.zeros((x_val_fold.size(0)))
        avg_val_loss = 0.
        for i, (x_batch, y_batch) in enumerate(valid_loader):
            y_pred = model(x_batch).detach()
            avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
            valid_preds_fold[i * batch_size:(i+1) * batch_size] = sigmoid(y_pred.cpu().numpy())[:, 0]
        auc = roc_auc_score(train.loc[test_index]['target'],valid_preds_fold)
        if auc >= best_auc:
            best_auc = auc
            torch.save(model.state_dict(), 'params.pkl')
        else:
            earlystop += 1
            if debug:
                print("epoch{}, auc{}".format(epoch,auc))
            if earlystop >=10:
                break
    avg_losses_f.append(avg_loss)
    avg_val_losses_f.append(avg_val_loss)
    print("Avg train loss:{} fold:{}".format(avg_loss,fold_))
    print("Avg valid loss:{} fold:{}".format(avg_val_loss,fold_))
    print("-"*50)
    model.load_state_dict(torch.load('params.pkl'))
    model.eval()
    for i, (x_batch,) in enumerate(test_loader):
        y_pred = model(x_batch).detach()
        test_preds_fold[i * batch_size:(i+1) * batch_size] = sigmoid(y_pred.cpu().numpy())[:, 0]
    valid_preds_fold = np.zeros((x_val_fold.size(0)))
    for i, (x_batch, y_batch) in enumerate(valid_loader):
        y_pred = model(x_batch).detach()
        valid_preds_fold[i * batch_size:(i+1) * batch_size] = sigmoid(y_pred.cpu().numpy())[:, 0]
    oof[idx1[test_index]] += valid_preds_fold#clf.predict_proba(train2.loc[test_index][cols])[:,1]
    preds[idx2] += test_preds_fold / 20 #clf.predict_proba(test2[cols])[:,1] / 25.0
    fold_ += 1
# PRINT CV AUC
auc = roc_auc_score(train['target'],oof)
print('CV =',round(auc,5))